In [1]:
!pip install geopandas

     |████████████████████████████████| 921kB 4.8MB/s 
     |████████████████████████████████| 10.1MB 34.0MB/s 
     |████████████████████████████████| 11.9MB 194kB/s 


In [2]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

PATH_TO_DATA = Path('drive/My Drive/sendy/data')
SEED = 42

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import geopandas as gpd
import shapely as shp
import pickle

In [0]:
train = pd.read_csv(PATH_TO_DATA / 'given/Train.csv')
test = pd.read_csv(PATH_TO_DATA / 'given/Test.csv')

In [8]:
train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [0]:
geo_cols = ['Pickup Lat', 'Pickup Long', 'Destination Lat', 'Destination Long']

In [0]:
geo = pd.concat((train[geo_cols], test[geo_cols])).reset_index(drop=True)

In [7]:
geo.shape, train.shape, test.shape

((28269, 4), (21201, 29), (7068, 25))

In [23]:
!pip install pycurl

     |████████████████████████████████| 215kB 5.0MB/s 
  Created wheel for pycurl: filename=pycurl-7.43.0.3-cp36-cp36m-linux_x86_64.whl size=282654 sha256=864348a95c206fa278957331b247380dc07377d28a2a379815cccf89f0a4a955
  Stored in directory: /root/.cache/pip/wheels/89/40/f5/7b4f2285aca871b5173887a6c69127210d92806c0d3a977e51
Successfully built pycurl


In [0]:
from urllib.request import Request
import urllib.request
from pycurl import Curl
import json

In [74]:
geo.head()

,Pickup Lat,Pickup Long,Destination Lat,Destination Long
0,-1.317755,36.830370,-1.300406,36.829741
1,-1.351453,36.899315,-1.295004,36.814358
2,-1.308284,36.843419,-1.300921,36.828195
3,-1.281301,36.832396,-1.257147,36.795063
4,-1.266597,36.792118,-1.295041,36.809817


In [0]:
coords = geo.apply(lambda x: [[x[1],x[0]],[x[3],x[2]]], axis=1)

In [9]:
coords[0]

[[36.8303703, -1.3177546999999998], [36.829741, -1.3004062]]

In [0]:
tokens = [
    '5b3ce3597851110001cf624858da346ad7ef476797bfa52b4f496330',
    '5b3ce3597851110001cf624827b3f32e448245109ef192cbfbf8d9c0',
    '5b3ce3597851110001cf624879aca553dda14064b5a46787c4a4556e',
    '5b3ce3597851110001cf6248e78d6ff3d63d4dacb646372f3559d5f1',
    '5b3ce3597851110001cf624858ebd0180efd4447953e21e0d2341b6b',
    '5b3ce3597851110001cf62486a9ed168a69a429fa58df6fdc3f89620',
    '5b3ce3597851110001cf6248726f8103f89443dd9535dfd250da3455'
]

In [8]:
 
with open(PATH_TO_DATA / 'preprocessed/coords.pkl', 'rb') as f:
    coords_info = pickle.load(f)

EOFError: ignored

In [5]:
len(coords_info)

0

In [15]:
import requests
import time
import pickle

i = len(coords_info)
for token in tokens:
    status_code = 200
    while status_code == 200 and len(coords_info) < len(coords): 
        body = {"coordinates":coords[i],
                "attributes":["avgspeed","detourfactor","percentage"],
                "elevation":"true",
                "instructions":"true","instructions_format":"text",
                "maneuvers":"true","preference":"recommended",
                "roundabout_exits":"true","suppress_warnings":"true",
                "units":"km","geometry":"true"}

        headers = {
            'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
            'Authorization': token
        }
        call = requests.post('https://api.openrouteservice.org/v2/directions/cycling-electric', json=body, headers=headers)

        if call.status_code == 200:
            coords_info.append(call.text)
            time.sleep(2)
            i += 1
            with open(PATH_TO_DATA / 'preprocessed/coords.pkl', 'wb') as f:
                pickle.dump(coords_info, f)
        else:
            print(f'End requests for token {token}')
            print(call.status_code, call.reason)
            status_code = call.status_code


End requests for token 5b3ce3597851110001cf624858da346ad7ef476797bfa52b4f496330
403 Forbidden


KeyboardInterrupt: ignored

In [0]:
import pickle

In [0]:
url = 'http://router.project-osrm.org/route/v1/driving/'
request_data = 'geometries=geojson&alternatives=true&steps=true&annotations=true&overview=full&continue_straight=true'

In [153]:
!pip install random-user-agent

     |████████████████████████████████| 8.2MB 4.7MB/s 


In [0]:
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100000)

# Get Random User Agent String.
user_agent = user_agent_rotator.get_random_user_agent()

In [162]:
lines = []
for i in range(len(coords)):
    user_agent = user_agent_rotator.get_random_user_agent()
    request = Request(url=url + f'{coords[i]}?{request_data}',
                  headers = 
                  {'User-agent': user_agent})
    with urllib.request.urlopen(request) as response:
        data = response.read()
    line.append(data)

HTTPError: ignored

In [163]:
lines

[]

In [0]:
with urllib.request.urlopen(request) as response:
   data = response.read()

In [131]:
url + f'{coords[4]}?{data}'

'http://project-osrm.org/route/v1/driving/-1.2665973999999998,36.7921181;-1.2950408000000002,36.809817200000005?geometries=geojson&alternatives=true&steps=true&annotations=true&overview=full&continue_straight=true'

In [93]:
json.loads(data)['routes'][0]['geometry']

{'coordinates': [[-1.879314, 37.01904],
  [-1.88292, 37.014793],
  [-1.889622, 37.010721],
  [-1.891998, 37.005529],
  [-1.890725, 37.001865],
  [-1.891475, 36.999838],
  [-1.892123, 36.999225],
  [-1.896731, 36.999823],
  [-1.896652, 36.996732],
  [-1.895523, 36.996037],
  [-1.899167, 36.992624],
  [-1.902127, 36.986838],
  [-1.902634, 36.98195],
  [-1.902103, 36.971201],
  [-1.901309, 36.967586],
  [-1.901808, 36.964094],
  [-1.90408, 36.960819],
  [-1.904385, 36.958251],
  [-1.903514, 36.955983],
  [-1.904563, 36.953128],
  [-1.905532, 36.952226],
  [-1.904683, 36.95104],
  [-1.904939, 36.94943],
  [-1.906262, 36.948143],
  [-1.906254, 36.946188],
  [-1.905365, 36.946049]],
 'type': 'LineString'}

In [0]:
line = shp.geometry.asLineString(json.loads(data)['routes'][0]['geometry']['coordinates'])

In [0]:
pol = line.convex_hull

In [97]:
pol.area

0.0005235079974999572

In [0]:
pol = line.

In [88]:
pol.coords[:]

[(13.392761678010935, 52.522984832450994)]

In [89]:
line.coords[:]

[(-1.905365, 36.946049), (-1.905365, 36.946049)]

In [98]:
line.length

0.09067299632402123